In [2]:
import temporal_graph
import matplotlib.pyplot as plt
import temporal_graph_for_training


import importlib
import torch

import temporal_graph_for_training
importlib.reload(temporal_graph_for_training)

from temporal_graph_for_training import TemporalGraph, TemporalTruthModel


In [3]:
temporal_graph = temporal_graph_for_training.TemporalGraph()




creating time intervall ...
the time intervall of the data goes
from: 2022-02-08 16:22:00 
to: 2022-10-14 20:48:00
we have the following time intervalls[(Timestamp('2022-02-08 16:22:00'), Timestamp('2022-02-15 16:22:00')), (Timestamp('2022-02-15 16:22:00'), Timestamp('2022-02-22 16:22:00')), (Timestamp('2022-02-22 16:22:00'), Timestamp('2022-03-01 16:22:00')), (Timestamp('2022-03-01 16:22:00'), Timestamp('2022-03-08 16:22:00')), (Timestamp('2022-03-08 16:22:00'), Timestamp('2022-03-15 16:22:00')), (Timestamp('2022-03-15 16:22:00'), Timestamp('2022-03-22 16:22:00')), (Timestamp('2022-03-22 16:22:00'), Timestamp('2022-03-29 16:22:00')), (Timestamp('2022-03-29 16:22:00'), Timestamp('2022-04-05 16:22:00')), (Timestamp('2022-04-05 16:22:00'), Timestamp('2022-04-12 16:22:00')), (Timestamp('2022-04-12 16:22:00'), Timestamp('2022-04-19 16:22:00')), (Timestamp('2022-04-19 16:22:00'), Timestamp('2022-04-26 16:22:00')), (Timestamp('2022-04-26 16:22:00'), Timestamp('2022-05-03 16:22:00')), (Timest

In [4]:
# inspect raw author values
authors = temporal_graph.comment_df["author"].head(10).tolist()
print("Sample authors from CSV:", authors)
print("Type of first author:", type(authors[0]))

# inspect user node keys
keys = list(temporal_graph.user_nodes.keys())[:10]
print("Sample user node keys:", keys)
print("Type of key:", type(keys[0]))

for u in temporal_graph.user_nodes.values():
    if u.comments:
        print("FOUND COMMENTS")
        print(len(u.comments))
        break

Sample authors from CSV: [2247, 20054, 11225, 11225, 11225, 11225, 11225, 11225, 11225, 11225]
Type of first author: <class 'int'>
Sample user node keys: ['2247', '20054', '11225', '2335', '15010', '20030', '14695', '20058', '18270', '17712']
Type of key: <class 'str'>
FOUND COMMENTS
87


In [5]:
for u in temporal_graph.user_nodes.values():
    
    if u.comments:
        c = next(iter(u.comments.values()))
        break
else:
    raise RuntimeError("No users with comments found!")

In [6]:
print("===== SANITY CHECKS =====")
print("Number of users:", len(temporal_graph.user_nodes))
print("Number of time intervals:",
      len(temporal_graph.time_intervall.time_intervalls))

# count comments per interval (global)
interval_counts = [
    sum(len(u.comments_in_intervall[t])
        for u in temporal_graph.user_nodes.values())
    for t in range(len(temporal_graph.time_intervall.time_intervalls))
]

print("Comments per interval (first 10):", interval_counts[:10])
print("Total comments:", sum(interval_counts))
print("=========================\n")


===== SANITY CHECKS =====
Number of users: 2251
Number of time intervals: 36
Comments per interval (first 10): [433, 2394, 4558, 5140, 5657, 4444, 4016, 3663, 3238, 2720]
Total comments: 164705



In [ ]:
import torch
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import gc


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

COMMENT_HIDDEN = 128
USER_HIDDEN = 128

PLOTS_ROOT = "plots"
os.makedirs(PLOTS_ROOT, exist_ok=True)


# ------------------------------------------------------
# 1) Infer feature dimension
# ------------------------------------------------------
any_user = next(
    u for u in temporal_graph.user_nodes.values()
    if len(u.comments) > 0
)
any_comment = next(iter(any_user.comments.values()))
COMMENT_DIM = any_comment.feature_vec.shape[0]
print("COMMENT_DIM =", COMMENT_DIM)


# ------------------------------------------------------
# 2) Temporal split
# ------------------------------------------------------
T = len(temporal_graph.time_intervall.time_intervalls)

T_TRAIN = int(0.8 * T)
T_VAL   = int(1 * T)

train_ts = list(range(0, T_TRAIN))
val_ts   = list(range(T_TRAIN, T_VAL))

print(f"Train: 0 → {T_TRAIN-1}")
print(f"Val  : {T_VAL-1}")


# ------------------------------------------------------
# 3) Metrics
# ------------------------------------------------------
def compute_metrics(logits, labels, threshold):

    probs = torch.sigmoid(logits)
    preds = (probs > threshold).float()
    labels_bin = (labels >= 0.5).float()

    tp1 = ((preds == 1) & (labels_bin == 1)).sum().item()
    fp1 = ((preds == 1) & (labels_bin == 0)).sum().item()
    fn1 = ((preds == 0) & (labels_bin == 1)).sum().item()

    tp0 = ((preds == 0) & (labels_bin == 0)).sum().item()
    fp0 = ((preds == 0) & (labels_bin == 1)).sum().item()
    fn0 = ((preds == 1) & (labels_bin == 0)).sum().item()

    prec1 = tp1 / (tp1 + fp1 + 1e-8)
    rec1  = tp1 / (tp1 + fn1 + 1e-8)
    f1_1  = 2 * prec1 * rec1 / (prec1 + rec1 + 1e-8)

    prec0 = tp0 / (tp0 + fp0 + 1e-8)
    rec0  = tp0 / (tp0 + fn0 + 1e-8)
    f1_0  = 2 * prec0 * rec0 / (prec0 + rec0 + 1e-8)

    macro_f1 = (f1_0 + f1_1) / 2.0
    acc = (preds == labels_bin).float().mean().item()

    return {
        "precision_class_1": prec1,
        "recall_class_1": rec1,
        "precision_class_0": prec0,
        "recall_class_0": rec0,
        "f1_class_1": f1_1,
        "f1_class_0": f1_0,
        "macro_f1": macro_f1,
        "accuracy": acc
    }


# ------------------------------------------------------
# 4) Time-Series Plot Generator (NEW)
# ------------------------------------------------------
def generate_trial_plots(trial_id, history):

    trial_dir = os.path.join(PLOTS_ROOT, f"trial_{trial_id}")
    os.makedirs(trial_dir, exist_ok=True)

    epochs = range(1, len(history["loss"]) + 1)

    # Training Loss
    plt.figure()
    plt.plot(epochs, history["loss"])
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss Over Time")
    plt.tight_layout()
    plt.savefig(os.path.join(trial_dir, "loss_curve.png"))
    plt.close()

    # Precision
    plt.figure()
    plt.plot(epochs, history["precision_class_0"], label="Precision Class 0")
    plt.plot(epochs, history["precision_class_1"], label="Precision Class 1")
    plt.xlabel("Epoch")
    plt.ylabel("Precision")
    plt.legend()
    plt.title("Precision Over Time")
    plt.tight_layout()
    plt.savefig(os.path.join(trial_dir, "precision_curve.png"))
    plt.close()

    # Recall
    plt.figure()
    plt.plot(epochs, history["recall_class_0"], label="Recall Class 0")
    plt.plot(epochs, history["recall_class_1"], label="Recall Class 1")
    plt.xlabel("Epoch")
    plt.ylabel("Recall")
    plt.legend()
    plt.title("Recall Over Time")
    plt.tight_layout()
    plt.savefig(os.path.join(trial_dir, "recall_curve.png"))
    plt.close()

    # F1
    plt.figure()
    plt.plot(epochs, history["f1_class_0"], label="F1 Class 0")
    plt.plot(epochs, history["f1_class_1"], label="F1 Class 1")
    plt.plot(epochs, history["macro_f1"], label="Macro F1")
    plt.xlabel("Epoch")
    plt.ylabel("F1 Score")
    plt.legend()
    plt.title("F1 Development")
    plt.tight_layout()
    plt.savefig(os.path.join(trial_dir, "f1_curve.png"))
    plt.close()

    # Accuracy
    plt.figure()
    plt.plot(epochs, history["accuracy"])
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Accuracy Over Time")
    plt.tight_layout()
    plt.savefig(os.path.join(trial_dir, "accuracy_curve.png"))
    plt.close()

    print(f"Saved time-series plots to {trial_dir}")


# ------------------------------------------------------
# 5) Sweep configuration (UNCHANGED)
# ------------------------------------------------------
thresholds  = [0.35]
use_past_y_opts   = [False]
use_graph_opts    = [True]
use_comments_opts = [True]
graph_types = ["sage"]

EPOCHS = 50
PATIENCE = 5
LR = 5e-4

results = []
best_macro_f1 = -1
best_config = None
best_model_path = "best_model.pt"

trial_id = 0


# ------------------------------------------------------
# 6) Sweep (ONLY metric storage added)
# ------------------------------------------------------
for (thresh,
     use_past_y,
     use_graph,
    graph_types,
     use_comments) in itertools.product(
        thresholds,
        use_past_y_opts,
        use_graph_opts,
    graph_types,
        use_comments_opts
    ):

    if not use_comments and not use_past_y:
        continue

    trial_id += 1
    print(f"\n=== Trial {trial_id} ===")
    print(f"Threshold: {thresh}, Use Past Y: {use_past_y}, "
          f"Use Graph: {use_graph}, Use Comments: {use_comments}, Graph Type: {graph_types}")  
    
    model = TemporalTruthModel(
        COMMENT_DIM,
        use_past_y=use_past_y,
        use_graph=use_graph,
        use_comments=use_comments,
        graph_type=graph_types
    ).to(DEVICE)

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=LR,
        weight_decay=1e-4
    )

    criterion = torch.nn.BCEWithLogitsLoss()
    user_ids = list(temporal_graph.user_nodes.keys())

    best_trial_macro = -1
    patience_ctr = 0

    history = {
        "loss": [],
        "precision_class_0": [],
        "precision_class_1": [],
        "recall_class_0": [],
        "recall_class_1": [],
        "f1_class_0": [],
        "f1_class_1": [],
        "macro_f1": [],
        "accuracy": []
    }

    for epoch in range(EPOCHS):

        model.train()
        user_states = {u: torch.zeros(USER_HIDDEN, device=DEVICE)
                       for u in user_ids}

        total_loss = 0.0

        for t in train_ts:

            optimizer.zero_grad()

            logits, labels, user_states = model.forward_one_step(
                temporal_graph, t, user_states
            )

            loss = criterion(logits, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()

            for u in user_states:
                user_states[u] = user_states[u].detach()

        # Validation
        model.eval()
        with torch.no_grad():

            user_states = {u: torch.zeros(USER_HIDDEN, device=DEVICE)
                           for u in user_ids}

            val_logits_all = []
            val_labels_all = []

            for t in val_ts:
                logits, labels, user_states = model.forward_one_step(
                    temporal_graph, t, user_states
                )
                val_logits_all.append(logits)
                val_labels_all.append(labels)

            val_logits = torch.cat(val_logits_all)
            val_labels = torch.cat(val_labels_all)

            metrics = compute_metrics(val_logits, val_labels, thresh)

        # Store history
        history["loss"].append(total_loss)
        for k in metrics:
            history[k].append(metrics[k])

        macro_f1 = metrics["macro_f1"]

        print(f"Epoch {epoch:02d} | Loss: {total_loss:.3f} | "
              f"Macro F1: {macro_f1:.3f}| Acc: {metrics['accuracy']:.3f}"
              f"Prec1: {metrics['precision_class_1']:.3f}|Rec1: {metrics['recall_class_1']:.3f}|F1_1: {metrics['f1_class_1']:.3f}"
              f"|Prec0: {metrics['precision_class_0']:.3f}|Rec0: {metrics['recall_class_0']:.3f}|F1_0: {metrics['f1_class_0']:.3f}")

        if macro_f1 > best_trial_macro + 1e-4:
            best_trial_macro = macro_f1
            best_trial_metrics = metrics
            patience_ctr = 0
        else:
            patience_ctr += 1
            if patience_ctr >= PATIENCE:
                print("Early stopping.")
                break

    generate_trial_plots(trial_id, history)

    results.append({
        "trial": trial_id,
        "threshold": thresh,
        "use_past_y": use_past_y,
        "use_graph": use_graph,
        "use_comments": use_comments,
        **best_trial_metrics
    })

    if best_trial_macro > best_macro_f1:
        best_macro_f1 = best_trial_macro
        best_config = results[-1]
        torch.save(model.cpu().state_dict(), best_model_path)
        model.to(DEVICE)

    del model
    del optimizer
    torch.cuda.empty_cache()
    gc.collect()


# ------------------------------------------------------
# 7) Save results (UNCHANGED)
# ------------------------------------------------------
results_df = pd.DataFrame(results).sort_values(
    "macro_f1", ascending=False
)

results_df.to_csv("full_sweep_results.csv", index=False)

print("\n===== SWEEP COMPLETE =====")
print("Best configuration:")
print(best_config)
print("\nResults saved to full_sweep_results.csv")
print("Best model saved to best_model.pt")

Using device: cuda
COMMENT_DIM = 129
Train: 0 → 27
Val  : 35

=== Trial 1 ===
Threshold: 0.35, Use Past Y: False, Use Graph: False, Use Comments: True, Graph Type: sage
Epoch 00 | Loss: 17.228 | Macro F1: 0.290| Acc: 0.317Prec1: 0.082|Rec1: 0.957|F1_1: 0.151|Prec0: 0.989|Rec0: 0.274|F1_0: 0.429
Epoch 01 | Loss: 9.894 | Macro F1: 0.557| Acc: 0.716Prec1: 0.173|Rec1: 0.918|F1_1: 0.291|Prec0: 0.992|Rec0: 0.703|F1_0: 0.823
Epoch 02 | Loss: 8.450 | Macro F1: 0.539| Acc: 0.684Prec1: 0.164|Rec1: 0.970|F1_1: 0.280|Prec0: 0.997|Rec0: 0.665|F1_0: 0.798
Epoch 03 | Loss: 7.940 | Macro F1: 0.525| Acc: 0.662Prec1: 0.156|Rec1: 0.985|F1_1: 0.269|Prec0: 0.998|Rec0: 0.640|F1_0: 0.780
Epoch 04 | Loss: 7.421 | Macro F1: 0.625| Acc: 0.797Prec1: 0.231|Rec1: 0.946|F1_1: 0.371|Prec0: 0.995|Rec0: 0.787|F1_0: 0.879
Epoch 05 | Loss: 6.360 | Macro F1: 0.641| Acc: 0.819Prec1: 0.247|Rec1: 0.910|F1_1: 0.389|Prec0: 0.993|Rec0: 0.813|F1_0: 0.894
Epoch 06 | Loss: 5.955 | Macro F1: 0.644| Acc: 0.819Prec1: 0.250|Rec1: 0.9

In [8]:
# ==========================================
# TEMPORAL CLUSTERING + INFLUENCE ANALYSIS
# WITH PLOTS
# ==========================================

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

T = len(temporal_graph.time_intervall.time_intervalls)
user_ids = list(temporal_graph.user_nodes.keys())

# -----------------------------------------
# 1️⃣ BUILD STATIC GRAPH
# -----------------------------------------
G = nx.DiGraph()

for u in user_ids:
    G.add_node(u)

for u_id, node in temporal_graph.user_nodes.items():
    for neigh in node.outgoing_edges:
        G.add_edge(u_id, str(neigh.user_id))


# -----------------------------------------
# 2️⃣ BUILD DANGEROUS MATRIX
# -----------------------------------------
danger = {
    u: [
        1 if flag == 1.0 else 0
        for flag in temporal_graph.user_nodes[u].truth_flag_intervall
    ]
    for u in user_ids
}

# ==========================================
# STORAGE ARRAYS
# ==========================================
danger_counts = []
densities = []
largest_components = []
assortativities = []
d2d_ratios = []
risk_ratios = []
risk_times = []

# ==========================================
# TEMPORAL CLUSTERING
# ==========================================

for t in range(T):

    dangerous_users_t = [
        u for u in user_ids if danger[u][t] == 1
    ]

    danger_counts.append(len(dangerous_users_t))

    if len(dangerous_users_t) == 0:
        densities.append(0)
        largest_components.append(0)
        assortativities.append(0)
        d2d_ratios.append(0)
        continue

    G_t = G.subgraph(dangerous_users_t)

    density = nx.density(G_t)
    components = list(nx.weakly_connected_components(G_t))
    largest_component = max(len(c) for c in components)

    attr = {u: danger[u][t] for u in user_ids}
    nx.set_node_attributes(G, attr, "danger_t")
    assort = nx.attribute_assortativity_coefficient(G, "danger_t")

    d2d = 0
    total_edges = 0
    for u, v in G.edges():
        if danger[u][t] == 1 and danger[v][t] == 1:
            d2d += 1
        total_edges += 1

    ratio = d2d / total_edges if total_edges > 0 else 0

    densities.append(density)
    largest_components.append(largest_component)
    assortativities.append(assort)
    d2d_ratios.append(ratio)


# ==========================================
# TEMPORAL INFLUENCE
# ==========================================

for t in range(T - 1):

    became_with_exposure = 0
    total_with_exposure = 0

    became_without_exposure = 0
    total_without_exposure = 0

    for u in user_ids:

        if danger[u][t] == 0:

            node = temporal_graph.user_nodes[u]

            exposed = False
            for neigh in node.ingoing_edges:
                v = str(neigh.user_id)
                if danger[v][t] == 1:
                    exposed = True
                    break

            became_danger = danger[u][t+1] == 1

            if exposed:
                total_with_exposure += 1
                if became_danger:
                    became_with_exposure += 1
            else:
                total_without_exposure += 1
                if became_danger:
                    became_without_exposure += 1

    if total_with_exposure == 0 or total_without_exposure == 0:
        continue

    p_exp = became_with_exposure / total_with_exposure
    p_noexp = became_without_exposure / total_without_exposure

    risk_ratio = p_exp / p_noexp if p_noexp > 0 else np.nan

    risk_ratios.append(risk_ratio)
    risk_times.append(t)

# ==========================================
#  PUBLICATION-READY PLOTS
# ==========================================

plt.rcParams.update({
    "font.size": 12,
    "axes.labelsize": 13,
    "axes.titlesize": 14,
    "legend.fontsize": 11,
    "figure.figsize": (8, 5)
})

# ------------------------------------------
# 1️⃣ Number of Untruthful Users
# ------------------------------------------
plt.figure()
plt.plot(range(T), danger_counts, marker="o")
plt.xlabel("Time Interval")
plt.ylabel("Number of Untruthful Users")
plt.title("Temporal Evolution of Untruthful Users")
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.savefig("untruthful_users_over_time.png", dpi=300)
plt.close()


# ------------------------------------------
# 2️⃣ Clustering Metrics
# ------------------------------------------
plt.figure()
plt.plot(range(T), assortativities, marker="o", label="Assortativity")
plt.plot(range(T), d2d_ratios, marker="s", label="Fraction of Edges Between Untruthful Users")
plt.xlabel("Time Interval")
plt.ylabel("Metric Value")
plt.title("Structural Concentration of Untruthful Users")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.savefig("clustering_metrics_over_time.png", dpi=300)
plt.close()


# ------------------------------------------
# 3️⃣ Risk Ratio
# ------------------------------------------
plt.figure()
plt.plot(risk_times, risk_ratios, marker="o")
plt.axhline(y=1.0, linestyle="--")  # baseline
plt.xlabel("Transition (t → t+1)")
plt.ylabel("Risk Ratio")
plt.title("Exposure-Based Risk Ratio Over Time")
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.savefig("risk_ratio_over_time.png", dpi=300)
plt.close()

print("Improved plots saved successfully.")



Improved plots saved successfully.


In [9]:
# ======================================================
# 7) VISUALIZATIONS
# ======================================================

import matplotlib.pyplot as plt
import numpy as np

# Reload results (optional safety)
results_df = pd.read_csv("full_sweep_results.csv")

# ------------------------------------------------------
# Plot 1 — Macro F1 per Trial
# ------------------------------------------------------
plt.figure()
plt.bar(range(len(results_df)), results_df["best_macro_f1"])
plt.xlabel("Trial")
plt.ylabel("Best Macro F1")
plt.title("Macro F1 per Trial")
plt.xticks(range(len(results_df)))
plt.tight_layout()
plt.savefig("plot_macro_f1_per_trial.png")
plt.show()


# ------------------------------------------------------
# Plot 2 — Macro F1 by Threshold
# ------------------------------------------------------
plt.figure()

thresholds = sorted(results_df["threshold"].unique())
means = []

for th in thresholds:
    subset = results_df[results_df["threshold"] == th]
    means.append(subset["best_macro_f1"].mean())

plt.plot(thresholds, means)
plt.xlabel("Threshold")
plt.ylabel("Average Macro F1")
plt.title("Average Macro F1 vs Threshold")
plt.tight_layout()
plt.savefig("plot_macro_f1_by_threshold.png")
plt.show()


# ------------------------------------------------------
# Plot 3 — Class-wise F1 for Best Trial
# ------------------------------------------------------

# Identify best config
best_row = results_df.iloc[0]

print("Best row for plotting:", best_row)

# Re-load best model metrics from last validation run
# (Assumes you printed per-class F1 for the best trial)

# If you want exact per-class F1 stored,
# you should also store them during sweep.
# For now, we assume you manually note them.

# Example placeholders (replace if you store actual values)
f1_class_1 = best_trial_macro  # Replace properly if saved
f1_class_0 = best_trial_macro  # Replace properly if saved

plt.figure()
plt.bar(["Class 0", "Class 1"], [f1_class_0, f1_class_1])
plt.ylabel("F1 Score")
plt.title("Class-wise F1 (Best Model)")
plt.tight_layout()
plt.savefig("plot_classwise_f1_best.png")
plt.show()


print("Plots saved:")
print("- plot_macro_f1_per_trial.png")
print("- plot_macro_f1_by_threshold.png")
print("- plot_classwise_f1_best.png")


KeyError: 'best_macro_f1'

<Figure size 800x500 with 0 Axes>

In [ ]:
# ================================
# TEMPORAL TRAIN / TEST + F1 SCORE
# ================================

import torch

# -------------------------------------------------------
# 1) Determine comment feature dimensionality
# -------------------------------------------------------
any_user = next(
    u for u in temporal_graph.user_nodes.values()
    if len(u.comments) > 0
)
any_comment = next(iter(any_user.comments.values()))
COMMENT_DIM = any_comment.feature_vec.shape[0]
print("COMMENT_DIM =", COMMENT_DIM)


# -------------------------------------------------------
# 2) Device
# -------------------------------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)


# -------------------------------------------------------
# 3) Temporal split (by time interval)
# -------------------------------------------------------
T = len(temporal_graph.time_intervall.time_intervalls)

TRAIN_RATIO = 0.8
T_TRAIN = int(T * TRAIN_RATIO)

train_timesteps = list(range(0, T_TRAIN))
test_timesteps  = list(range(T_TRAIN, T))

print(f"Train timesteps: 0 → {T_TRAIN-1}")
print(f"Test timesteps:  {T_TRAIN} → {T-1}")


# -------------------------------------------------------
# 4) Initialize model
# -------------------------------------------------------
print("Initializing model with use_past_y = True")
model = TemporalTruthModel(
    COMMENT_DIM,
    use_past_y=True
).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.BCEWithLogitsLoss()


# -------------------------------------------------------
# 5) Helper: accuracy + F1
# -------------------------------------------------------
def compute_metrics(logits, labels, threshold=0.5):
    probs = torch.sigmoid(logits)
    preds = (probs > threshold).float()

    tp = ((preds == 1) & (labels == 1)).sum().item()
    fp = ((preds == 1) & (labels == 0)).sum().item()
    fn = ((preds == 0) & (labels == 1)).sum().item()

    precision = tp / (tp + fp + 1e-8)
    recall    = tp / (tp + fn + 1e-8)
    f1        = 2 * precision * recall / (precision + recall + 1e-8)

    acc = (preds == labels).float().mean().item()

    return acc, precision, recall, f1


# -------------------------------------------------------
# 6) Training loop (TRAIN ONLY on early timesteps)
# -------------------------------------------------------
EPOCHS = 20
print("\nStarting training for", EPOCHS, "epochs...\n")

for epoch in range(EPOCHS):
    model.train()
    optimizer.zero_grad()

    logits_all, labels_all = model(temporal_graph)

    # ---- TRAIN LOSS (only train timesteps) ----
    train_loss = sum(
        criterion(logits_all[t], labels_all[t])
        for t in train_timesteps
    )

    train_loss.backward()
    optimizer.step()

    # ---- TRAIN METRICS ----
    with torch.no_grad():
        train_logits = torch.cat([logits_all[t] for t in train_timesteps])
        train_labels = torch.cat([labels_all[t] for t in train_timesteps])
        acc, p, r, f1 = compute_metrics(train_logits, train_labels)

    print(
        f"Epoch {epoch:02d} | "
        f"Train loss: {train_loss.item():.4f} | "
        f"Acc: {acc:.3f} | "
        f"F1: {f1:.3f}"
    )


# -------------------------------------------------------
# 7) Evaluation on TEST timesteps
# -------------------------------------------------------
model.eval()

with torch.no_grad():
    logits_all, labels_all = model(temporal_graph)

    test_logits = torch.cat([logits_all[t] for t in test_timesteps])
    test_labels = torch.cat([labels_all[t] for t in test_timesteps])

    acc, p, r, f1 = compute_metrics(test_logits, test_labels)

print("\n===== TEST RESULTS =====")
print(f"Accuracy : {acc:.3f}")
print(f"Precision: {p:.3f}")
print(f"Recall   : {r:.3f}")
print(f"F1 score : {f1:.3f}")
print("========================\n")


# -------------------------------------------------------
# 8) Save model
# -------------------------------------------------------
torch.save(model.state_dict(), "temporal_truth_model.pt")
print("Model saved to temporal_truth_model.pt")



In [ ]:
len(graph.comment_df)

In [ ]:
graph = temporal_graph.TemporalGraph()
graph.clean_nodes(True, True, False)


In [ ]:
import random
dangerous_users=[6609,211,4273,8242,1772,6559,9365,3321,4248,8759,455,9264,4869,4210,8760,5807,805,8751,8256,6157]
for dangerous_user in dangerous_users:
    print(graph.nodes_dict[str(dangerous_user)].username)
# value = random.choice(list(graph.nodes_dict.values()))

value = graph.nodes_dict["6609"]
value.node_info()


In [ ]:
graph.find_truth("807621")

In [ ]:
for truth_id in graph.nodes_dict["20054"].truths:
    print(type(truth_id))


In [ ]:
graph.create_pyvis_representation()


In [ ]:
following_edges_per_node = []
follower_edges_per_node = []
messages_per_node = []

counter_follower = 0
counter_following = 0
counter_message = 0

zero_counter_following = 0
zero_counter_follower = 0
zero_counter_message = 0

for key in graph.nodes_dict.keys():
    if len(graph.nodes_dict[key].following_edges) != 0:
        following_edges_per_node.append(len(graph.nodes_dict[key].following_edges))
        counter_following += 1
    else:
        zero_counter_following += 1

    if len(graph.nodes_dict[key].truths) != 0:
        messages_per_node.append(len(graph.nodes_dict[key].truths))
        counter_message += 1
    else:
        zero_counter_message += 1

    if len(graph.nodes_dict[key].follower_edges) != 0:
        follower_edges_per_node.append(len(graph.nodes_dict[key].follower_edges))
        counter_follower += 1
    else:
        zero_counter_follower += 1

print(f"nodes with no known followings: {zero_counter_following} nodes with followings:{counter_following}")
print(f"nodes with no known messages: {zero_counter_message} with known messages:{counter_message}")
print(f"nodes with no known followers: {zero_counter_follower} nodes with followers:{counter_follower}")


NameError: name 'graph' is not defined

In [ ]:
import numpy as np

print(np.median(following_edges_per_node))
plt.hist(following_edges_per_node, bins=3000)
plt.xscale("log")
plt.yscale("log")


In [ ]:
plt.hist(follower_edges_per_node, bins=3000)
plt.xscale("log")
plt.yscale("log")


In [ ]:
plt.hist(messages_per_node, bins=3000)
plt.xscale("log")
plt.yscale("log")
